
## Scripts QUBO 
Este cuaderno, reune código básico para ejecutar problemas QUBO (Quadratic Unconstrained Binary Optimization) en ordenadores cuánticos de annealing, como D-Wave. Se utilizan las bibliotecas `pyqubo` y `neal` para definir y resolver estos problemas. 

A continuación, se muestra un ejemplo de cómo definir un problema QUBO, compilarlo, convertirlo a un modelo binario cuadrático y resolverlo utilizando un simulador de annealing simulado.

In [ ]:
import neal
from pyqubo import Binary

a, b = Binary('a'), Binary('b')
H = 2*a +3*b*a -2*(a*b+1)**2
model = H.compile()
bqm = model.to_bqm()
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=10)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample)
print(best_sample.energy)
print(best_sample.constraints())

## Problema 1.1: Máximo Corte (Max-Cut) para un Grafo Pequeño

**Enunciado:**

Dado un grafo no dirigido $G = (V, E)$, donde $V$ es el conjunto de vértices y $E$ es el conjunto de aristas, encontrar una partición de los vértices en dos conjuntos $S$ y $V \setminus S$ tal que el número de aristas que cruzan la partición (es decir, que tienen un extremo en $S$ y el otro en $V \setminus S$) sea máximo.

**Instancia Específica:**

Considera un grafo simple con 4 vértices (A, B, C, D) y 5 aristas:

*   A - B
*   B - C
*   C - D
*   D - A
*   D - B

Este grafo es un ciclo de 4 vértices con una cuerda (arista A-C).

**Objetivo:**

1.  **Definir las variables binarias:** Asigna una variable binaria $x_i$ a cada vértice $i$, donde $x_i = 1$ si el vértice $i$ pertenece al conjunto $S$, y $x_i = 0$ si pertenece al conjunto $V \setminus S$.

2.  **Formular el problema como un QUBO:** Crea la matriz $Q$ del QUBO que representa la función objetivo a minimizar. Recuerda que el objetivo de Max-Cut se puede transformar en un problema de minimización equivalente.

3.  **Implementar el QUBO:** Utiliza un solver QUBO (por ejemplo, D-Wave Ocean SDK, Qiskit, SimAnneal, o Gurobi/CPLEX) para encontrar la asignación de variables que minimiza la función objetivo.

4.  **Interpretar los resultados:** Determina la partición de los vértices en los conjuntos $S$ y $V \setminus S$ basándote en la asignación de variables obtenida del solver.

5.  **Calcular el tamaño del corte:** Calcula el número de aristas que cruzan la partición encontrada.

**Consideraciones:**

*   Existen diferentes formulaciones QUBO para Max-Cut. Investiga y elige la que te parezca más adecuada.
*   El rendimiento del solver puede depender de la estructura específica del grafo y de los parámetros del solver.

**Sugerencias:**

*   Empieza escribiendo la función objetivo de Max-Cut en términos de las variables binarias $x_i$.
*   Luego, manipula la función objetivo para que tenga la forma estándar de un QUBO:  $\sum_{i} Q_{ii} x_i + \sum_{i<j} Q_{ij} x_i x_j$.
*   Verifica que la matriz $Q$ sea simétrica.

In [ ]:
import neal
from pyqubo import Spin
import numpy as np

v = 4
e = 5

V = np.empty(v, dtype=object) 

# Introducimos las variables en el diccionario. f'x{i}' es un f-string, genera una cadena de valor 'x' seguido de i.
for i in range(v):
    V[i]= Spin(f'x{i+1}')
    
Q = np.array([[0, 1, 0, 1],
              [1, 0, 1, 1],
              [0, 1, 0, 1],
              [1, 1, 1, 0]])

H=0
    
for i in range(v):
    for j in range(v):
        if i<j:
            H += -Q[i][j] + Q[i][j]*V[i]*V[j]
            
model = H.compile()
result = model.to_ising()

#print("Contenido de 'result':", result)

# Usa el diccionario devuelto por to_ising()
linear = result[0] # Bias lineal
ising = result[1]  # Interacciones
offset = result[2] # Offset

#Ahora puedes usar ising, linear y offset como necesites
print("Interacciones (J):", ising)
print("Bias Lineal (h):", linear)
print("Offset:", offset)

# Resolver con Simulated Annealing
sampler = neal.SimulatedAnnealingSampler() # minimiza la energia del problema que se le proporcina. Si quieres maximizar f(x), minimiza -f(x)

# Como linear (result[0]) es un diccionario vacío, necesitamos crear un diccionario de bias lineales vacío.
sampleset = sampler.sample_ising(linear, ising, num_reads=100) #Ejecuta el algoritmo de optimización en el problema. Devuelve un conjunto de muestras encontradas de minima energia en las 100 veces que se ejecuta.

decoded_samples = model.decode_sampleset(sampleset)# decoded_samples contiene información similar a sampleset, pero en un formato más fácil de entender y usar

# Analizar las soluciones decodificadas
for decoded_sample in decoded_samples:
    print("Sample:", decoded_sample.sample)  # Asignación de variables (x1=1, x2=0, ...)
    print("Energy:", decoded_sample.energy)
    print("Constraints:", decoded_sample.constraints(only_broken=True))  # Solo las restricciones violadas
    
# Obtener la mejor solución
best_sample = min(decoded_samples, key=lambda d: d.energy) #min(iterable, key=func) fuc= lambda d: d.energy función anonima que se aplica a cada elemento d. devuelve la energia de cada elemeto.

print(best_sample)
            
            


Interacciones (J): {('x1', 'x2'): 1.0, ('x2', 'x4'): 1.0, ('x1', 'x4'): 1.0, ('x2', 'x3'): 1.0, ('x3', 'x4'): 1.0}
Bias Lineal (h): {}
Offset: -5.0
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1, 'x2': -1, 'x3': 1, 'x4': -1}
Energy: -8.0
Constraints: {}
Sample: {'x1': 1